<a href="https://colab.research.google.com/github/ragalaa/Machine-Learning-Basics/blob/master/Recommendation_Engine_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Recommendation Engine from scrath using the Movielens Dataset

In [0]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
plt.style.use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Cosine Calculations as Prework for identifying:

In [0]:
from scipy import spatial
import numpy as np

In [0]:
a = np.array([2.0,1.0])
b = np.array([5.0,3.0])

In [47]:
1-spatial.distance.cosine(a,b)

0.9970544855015815

In [48]:
c= np.array([5.0,4.0])
1-spatial.distance.cosine(c,b)

0.9909924304103233

In [50]:
# Normal Cosine similarirty doesn't provide the differencer.
## Consine only provides direction od vectors in n dimension to find positions we need
# Adjacent Cosine similarity would give us a better understanding of resemblence of users.

mean_ab = sum(sum(a,b))  / 4
mean_ab


3.5

In [51]:
### Adjacent vectors  equal 
Adj_a = a - mean_ab
Adj_b = b- mean_ab
Adj_a,Adj_b

(array([-1.5, -2.5]), array([ 1.5, -0.5]))

In [52]:
1-spatial.distance.cosine(Adj_a,Adj_b)

-0.21693045781865616

In [0]:
mean_cb = sum(sum(c,b)) /4
Adj_C = c- mean_cb

In [55]:
1- spatial.distance.cosine(c - mean_cb,b-mean_cb)  ### This shows difference.

0.9908301680442989

# Actual Implementation of recommended systems:

In [2]:
## Users data set:

# !curl -O https://raw.githubusercontent.com/gonzaferreiro/Networks_Analysis_plus_Recommendations_system/master/ml-100k/u.user
##Ratings datset
# !curl -O https://raw.githubusercontent.com/gonzaferreiro/Networks_Analysis_plus_Recommendations_system/master/ml-100k/u.data
### Items datset
# !curl -O https://raw.githubusercontent.com/gonzaferreiro/Networks_Analysis_plus_Recommendations_system/master/ml-100k/u.item

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22628  100 22628    0     0   103k      0 --:--:-- --:--:-- --:--:--  103k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1932k  100 1932k    0     0  7377k      0 --:--:-- --:--:-- --:--:-- 7377k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  230k  100  230k    0     0  1154k      0 --:--:-- --:--:-- --:--:-- 1154k


In [0]:
# Reading file and passing columns names

col_users = ['user_id','age','sex','occupation','zip_code']
users = pd.read_csv('./u.user',sep='|', names=col_users)

col_rat = ['user_id','movie_id','rating','unix_timestamp']
ratings = pd.read_csv('./u.data',sep='\t', names=col_rat)

col_items = ['movie_id', 'movie_title' ,'release_date','video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('./u.item',sep='|', names=col_items, encoding='latin-1')

EDA ( Exploratory data analysis) with shape and **duplicates**

In [4]:
print('users'+ str(users.shape))
users.head()


users(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [5]:
# Checking duplicates
print((users.shape[0] - len([x for x in (users.user_id.value_counts() > 1).values if x == False]))/2)

0.0


In [6]:
print('Ratings'+ str(ratings.shape))
ratings.head()

Ratings(100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
print('Items'+ str(items.shape))
items.head()

Items(1682, 24)


,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [8]:
# Checking non repeated-movies
print((items.shape[0] - len([x for x in (items.movie_id.value_counts() > 1).values if x == False]))/2)

0.0


In [17]:
# First we need to crete a user-item matrix to be used 
# to calculate the similarity between users and items.

n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]
print( n_users, n_items)

# We'll create a matrix full of zeros
# And then we'll go by everyline in ratings, filling the user row
# and the movie column, with the rating.

data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

943 1682


In [0]:
# Now, we will calculate the similarity. 
# Regular Cosine Similarity by the definition reflects differences 
# in direction, but not the location. Therefore it does not consider 
# the difference in ratings of users. 
# Adjusted cosine similarity offsets this drawback by subtracting 
# the average rating.
# So we'll create a function to find the Adjusted Cosine Similarity:

from scipy import spatial

def adjusted_cos_distance_matrix(size, matrix, row_column):
    distances = np.zeros((size,size))
    if row_column == 0:
        M_u = matrix.mean(axis=1)
        m_sub = matrix - M_u[:,None]
    if row_column == 1:
        M_u = matrix.T.mean(axis=1)
        m_sub = matrix.T - M_u[:,None]
    for first in range(0,size):
        for sec in range(0,size):
            distance = spatial.distance.cosine(m_sub[first],m_sub[sec])
            distances[first,sec] = distance
    return distances

In [34]:
user_similarity = adjusted_cos_distance_matrix(n_users,data_matrix,0)
item_similarity = adjusted_cos_distance_matrix(n_items,data_matrix,1)





AttributeError: ignored

In [0]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating 
        # has same format as ratings. Find more about np.newaxis here:
        # https://stackoverflow.com/questions/29241056/how-does-numpy-newaxis-work-and-when-to-use-it
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred




In [0]:
# user_prediction = predict(data_matrix, user_similarity, type='user')
# item_prediction = predict(data_matrix, item_similarity, type='item')


In [0]:
# user_prediction.shape

In [0]:
# item_prediction.shape

In [0]:
########################




###########

# The following function uses the predictions previously created,
# finds the movies not rated by the user (and therefore not seen),
# a passes the top 5 ranked movies according to our predictions.

def finding_movies_for_user(user, ratings, preds):
    recos = {'movie_id': [],'rating': []}
    user_array = preds[user]
    orig_rat = ratings[user]
    for each in range(0,len(orig_rat)):
        if orig_rat[each] == 0:
            recos['movie_id'].append(each+1)
            recos['rating'].append(user_array[each])
    recos_df = pd.DataFrame(recos)
    final_recos_df = pd.merge(recos_df, items[['movie_id','movie_title']],
                              how='left', on='movie_id')
    return final_recos_df.sort_values(by='rating',ascending=False).head(5)

In [0]:

# finding_movies_for_user(10, data_matrix, user_prediction)